In [1]:
import matplotlib
%matplotlib notebook

In [17]:
import os
import datetime
import stompy.model.delft.waq_scenario as waq

In [18]:
import six
six.moves.reload_module(waq)

<module 'stompy.model.delft.waq_scenario' from '/home/rusty/python/stompy/model/delft/waq_scenario.py'>

In [19]:
agg_grid_shp="Agg2Shapefile_edited/agg_shapefile_edited.shp"
hyd_fn="/hpcvol1/public/sfb_dfm_v2/wy2013c/DFM_DELWAQ_wy2013c_adj/wy2013c.hyd"
output_fn="wy2013c-agg/wy2013c-agg.hyd"

In [9]:
# Clean inputs
def clean_shapefile(shp_in):
    from stompy.spatial import wkb2shp
    geoms=wkb2shp.shp2geom(agg_grid_shp)

    multi_count=0
    
    new_geoms=[]
    for fi,feat in enumerate(geoms):
        if feat['geom'].type=='Polygon':
            new_geoms.append(feat['geom'])
        else:
            multi_count+=1
            for g in feat['geom'].geoms:
                new_geoms.append(g)
    if multi_count:
        cleaned=agg_grid_shp.replace('.shp','-cleaned.shp')
        assert cleaned!=agg_grid_shp
        wkb2shp.wkb2shp(cleaned,new_geoms,overwrite=True)

        return cleaned
    else:
        return shp_in

shp=clean_shapefile(agg_grid_shp)    

In [20]:
hydro_orig=waq.HydroFiles(hyd_fn)

hydro_agg=waq.HydroAggregator(hydro_in=hydro_orig,
                              agg_shp=shp,
                              sparse_layers=False,
                              agg_boundaries=False)

INFO:HydroAggregator:Trying to triage bad dimensions in NetCDF (probably ddcouplefm output)
INFO:HydroAggregator:Synthesizing multi domain data for single domain run
INFO:HydroAggregator:init_elt_mapping: proc=0
INFO:HydroAggregator:Cell area missing in netcdf, will be computed from grid
INFO:HydroAggregator:2D element within aggregation polygon: 0
INFO:HydroAggregator:Calculating union of all aggregation polys
INFO:HydroAggregator:2D element within aggregation polygon: 2000
INFO:HydroAggregator:2D element within aggregation polygon: 4000
INFO:HydroAggregator:2D element within aggregation polygon: 6000
INFO:HydroAggregator:2D element within aggregation polygon: 8000
INFO:HydroAggregator:2D element within aggregation polygon: 10000
INFO:HydroAggregator:2D element within aggregation polygon: 12000
INFO:HydroAggregator:2D element within aggregation polygon: 14000
INFO:HydroAggregator:2D element within aggregation polygon: 16000
INFO:HydroAggregator:2D element within aggregation polygon: 1

----------------------------------------
Only showing first 10 elements


INFO:HydroFiles:Inferring 2D elements, 0 / 499960 3-D segments
INFO:HydroFiles:Inferring 2D elements, 50000 / 499960 3-D segments
INFO:HydroFiles:Inferring 2D elements, 100000 / 499960 3-D segments
INFO:HydroFiles:Inferring 2D elements, 150000 / 499960 3-D segments
INFO:HydroFiles:Inferring 2D elements, 200000 / 499960 3-D segments
INFO:HydroFiles:Inferring 2D elements, 250000 / 499960 3-D segments
INFO:HydroFiles:Inferring 2D elements, 300000 / 499960 3-D segments
INFO:HydroFiles:Inferring 2D elements, 350000 / 499960 3-D segments
INFO:HydroFiles:Inferring 2D elements, 400000 / 499960 3-D segments
INFO:HydroFiles:Inferring 2D elements, 450000 / 499960 3-D segments
INFO:HydroAggregator:Aggregated output will have
INFO:HydroAggregator: 150630 segments
INFO:HydroAggregator: 391557 exchanges (255990,0,135567)
INFO:HydroAggregator:  2480 boundary exchanges
INFO:HydroAggregator:150476 aggregated exchanges come from multiple unaggregated exchanges
INFO:HydroAggregator:had to choose internal 

In [ ]:
name=os.path.basename(output_fn.replace('.hyd',''))
class Writer(waq.Scenario):
    name=name
    desc=(name,
          agg_grid_shp,
          'aggregated')
    base_path=os.path.dirname(output_fn)

sec=datetime.timedelta(seconds=1)
if 0:
    # short run for testing: start after some hydro spinup:
    start_time=hydro_agg.time0+sec*hydro_agg.t_secs[100]
    # and run for 1.5 days..
    stop_time=start_time + 4*24*3600*sec
else:
    start_time=hydro_agg.time0+hydro_agg.t_secs[ 0]*sec
    stop_time =hydro_agg.time0+hydro_agg.t_secs[-1]*sec

writer=Writer(hydro=hydro_agg,
              start_time=start_time,
              stop_time=stop_time)

# This step is super slow.  Watch the output directory for progress.
# Takes hours on HPC
writer.cmd_write_hydro()

Setting hydro


INFO:HydroAggregator:Inferring 2D elements, 0 / 150630 3-D segments
INFO:HydroAggregator:Inferring 2D elements, 50000 / 150630 3-D segments
INFO:HydroAggregator:Inferring 2D elements, 100000 / 150630 3-D segments
INFO:HydroAggregator:Inferring 2D elements, 150000 / 150630 3-D segments
INFO:HydroFiles:Segment depth will be inferred


Incoming parameters are ['surf', 'bottomdept', 'vertdisper']
/hpcvol1/public/sfb_dfm_v2/wy2013c/DFM_DELWAQ_wy2013c_adj/wy2013c.vdf does exist, so will add it to the parameters
/hpcvol1/public/sfb_dfm_v2/wy2013c/DFM_DELWAQ_wy2013c_adj/wy2013c.tau does exist, so will add it to the parameters
/hpcvol1/public/sfb_dfm_v2/wy2013c/DFM_DELWAQ_wy2013c_adj/wy2013c.sal does exist, so will add it to the parameters


That ran, took about 16 hours.  Compared to the inputs, did not create a bed stress tau file.
 

In [33]:
hydro_orig.parameters()

INFO:HydroFiles:Segment depth will be inferred


Incoming parameters are ['surf', 'bottomdept', 'vertdisper']
/hpcvol1/public/sfb_dfm_v2/wy2013c/DFM_DELWAQ_wy2013c_adj/wy2013c.vdf does exist, so will add it to the parameters
/hpcvol1/public/sfb_dfm_v2/wy2013c/DFM_DELWAQ_wy2013c_adj/wy2013c.tau does exist, so will add it to the parameters
/hpcvol1/public/sfb_dfm_v2/wy2013c/DFM_DELWAQ_wy2013c_adj/wy2013c.sal does exist, so will add it to the parameters


NamedObjects([('surf',
               <stompy.model.delft.waq_scenario.ParameterSpatial at 0x7f99ad4ddc50>),
              ('bottomdept',
               <stompy.model.delft.waq_scenario.ParameterSpatial at 0x7f9927f4a630>),
              ('vertdisper',
               <stompy.model.delft.waq_scenario.ParameterSpatioTemporal at 0x7f991d938908>),
              ('tau',
               <stompy.model.delft.waq_scenario.ParameterSpatioTemporal at 0x7f991d938940>),
              ('salinity',
               <stompy.model.delft.waq_scenario.ParameterSpatioTemporal at 0x7f991d938860>)])